In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [3]:
import os

In [4]:
data_file = os.path.join(os.path.curdir, 'data', 'customer_churn.csv')

In [5]:
data_file

'./data/customer_churn.csv'

In [6]:
customer_data = spark.read.csv(data_file,
                              inferSchema=True,
                              header=True)

In [7]:
customer_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [8]:
customer_data.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [9]:
customer_data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [11]:
customer_data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
assembler = VectorAssembler(inputCols=[
    'Age',
    'Total_Purchase',
    'Account_Manager',
    'Years',
    'Num_Sites'], outputCol='features'
)

In [14]:
output = assembler.transform(customer_data)

In [15]:
output.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date='2013-08-30 07:00:40', Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1, features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]))

In [19]:
final_data = output.select('features', 'churn')

In [20]:
final_data.head()

Row(features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]), churn=1)

In [44]:
training_data, testing_data = final_data.randomSplit([0.8, 0.2])

In [45]:
training_data.count()

723

In [46]:
testing_data.count()

177

In [47]:
from pyspark.ml.classification import LogisticRegression

In [48]:
model = LogisticRegression(labelCol='churn')

In [49]:
predictor = model.fit(training_data)

In [50]:
predictor_summary = predictor.summary

In [51]:
predictor_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                723|                723|
|   mean|0.14937759336099585|0.10650069156293222|
| stddev|0.35670704616332694| 0.3086909335420211|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [52]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [53]:
preds = predictor.evaluate(testing_data)

In [54]:
preds.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[29.0,9378.24,0.0...|    0|[4.76895430146926...|[0.99158220804402...|       0.0|
|[30.0,13473.35,0....|    0|[2.68309934594494...|[0.93602197842554...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.62625895618555...|[0.97407445472305...|       0.0|
|[31.0,10058.87,1....|    0|[4.51340613616557...|[0.98915778058422...|       0.0|
|[32.0,7896.65,0.0...|    0|[3.57319135245953...|[0.97270006226475...|       0.0|
|[32.0,8011.38,0.0...|    0|[2.19649150729315...|[0.89993400434730...|       0.0|
|[32.0,9036.27,0.0...|    0|[0.10253834206152...|[0.52561214869767...|       0.0|
|[32.0,11715.72,0....|    0|[3.35278433136632...|[0.96619589410757...|       0.0|
|[32.0,12254.75,1....|    0|[2.63474192217363...|[0.93306431644576...|       0.0|
|[32.0,12403.6,0

In [55]:
prediction_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                     labelCol='churn')

In [57]:
auc = prediction_evaluator.evaluate(preds.predictions)

In [58]:
auc

0.7828042328042328

In [59]:
# predicting on unseen data now

In [60]:
total_input_data_model = model.fit(final_data)

In [61]:
new_data_file = os.path.join(os.path.curdir, 'data', 'new_customers.csv')

In [62]:
new_data = spark.read.csv(new_data_file, inferSchema=True, header=True)

In [63]:
test_new_customers = assembler.transform(new_data)

In [66]:
test_new_customers.head()

Row(Names='Andrew Mccall', Age=37.0, Total_Purchase=9935.53, Account_Manager=1, Years=7.71, Num_Sites=8.0, Onboard_date='2011-08-29 18:37:54', Location='38612 Johnny Stravenue Nataliebury, WI 15717-8316', Company='King Ltd', features=DenseVector([37.0, 9935.53, 1.0, 7.71, 8.0]))

In [67]:
final_results = total_input_data_model.transform(test_new_customers)

In [69]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168705251440...|[0.90218018099704...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595020...|[0.00198380445828...|       

In [72]:
final_results.select(['Company', 'prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

